Setup necessary packages

In [1]:
using DataFrames

Set the filename

In [2]:
# filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1805694.chadmin1.ib0.cheyenne.ucar.edu.200417-103849"
# filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1851933.chadmin1.ib0.cheyenne.ucar.edu.200420-102252"
# filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1855695.chadmin1.ib0.cheyenne.ucar.edu.200420-130249"
# filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1877502.chadmin1.ib0.cheyenne.ucar.edu.200421-125647"
filename = "/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1927093.chadmin1.ib0.cheyenne.ucar.edu.200423-153202"

"/home/gregorylemieux/Work/Issues/383/deciduous-griddedruns/cesm.log.1927093.chadmin1.ib0.cheyenne.ucar.edu.200423-153202"

Pull out the tread numbers (this should be sequential)cesm.log.1927093.chadmin1.ib0.cheyenne.ucar.edu.200423-153202

In [3]:
# Open the file
f = open(filename)

# Read through each line and pull out the unique threadnumbers
A = Vector{Int64}() # Initialize
for line in eachline(f)
    threadnum = parse(Int64,split(line,":")[1]) # Parse the line for the thread number
    if !(in(threadnum,A)) # If not in the array add it
        push!(A,threadnum)
    end
end

# Close the file
close(f)

Run a couple of checks

In [4]:
# Check all entries are unique
println(A == unique(A))

# Check that the thread numbers are sequential
println(sort!(A) == Array(0:maximum(A)))

true
true


## Setting up search values and dicts

In [5]:
# Set the known strings we are interested in
string_canopy = "Starting canopy trim"
string_cohort = "Current cohort"
string_laimem = "Starting laimemory"
string_leafm = "Starting leaf biomass"
string_treelai = "currentCohort%treelai"
string_season = "EDPftvarcon_inst%season_decid"
string_stress = "EDPftvarcon_inst%stress_decid"
string_evergreen = "EDPftvarcon_inst%evergreen"
string_optimum = "OPTIMUM_LAIMEM"
string_cumulative = "CUMULATIVE LAI"
string_trimopt = "OPTIMUM TRIM"
string_nnu = "nnu_clai_a"
string_nv = "currentCohort%nv"
string_cstatus = "currentSite%cstatus"
string_dstatus = "currentSite%dstatus"
string_coh = "currentCohort%status_coh"
string_end = "End of cohort info"

nll = 4

# Initialize vectors
thread_vec = Vector{Int64}()
laimem_vec = Vector{Float64}()
treelai_vec = Vector{Float64}()
leafmass_vec = Vector{Float64}()
leafmass_vec = Vector{Float64}()
seasonal_vec = Vector{Float64}()
stress_vec = Vector{Float64}()
evergreen_vec = Vector{Float64}()
leafonoff_vec = Vector{Int64}()
cstatus_vec = Vector{Int64}()
dstatus_vec = Vector{Int64}()

totallaicount = 0;

## Leaf Status Check

In [6]:
# Leaf status check loop
totallaicount = 0;
for t in A
    
    f = open(filename) # Open the file`
    
    # Iterate through file
    for line in eachline(f)
        
        linesplit = split(line,":") # Split into array of strings
        threadnum = parse(Int64,linesplit[1]) # Parse the line for the thread number
        
        # Make sure the line format is threadnum:string:val
        if threadnum == t && length(linesplit) == 3 && isempty(findall(isempty,linesplit))
            
            if strip(linesplit[2]) == string_laimem
                totallaicount += 1  # Count up the total laimemory loops
                push!(thread_vec,threadnum)
                push!(laimem_vec,parse(Float64,strip(linesplit[3])))
                
            # Next is the leaf biomass
            elseif strip(linesplit[2]) == string_leafm
                push!(leafmass_vec,parse(Float64,strip(linesplit[3])))

            # Next is the leaf biomass
            elseif strip(linesplit[2]) == string_treelai
                push!(treelai_vec,parse(Float64,strip(linesplit[3])))
                    
            # Next three lines should be the deciduous/evergreen states
            elseif strip(linesplit[2]) == string_season
                push!(seasonal_vec,parse(Float64,strip(linesplit[3])))
            elseif strip(linesplit[2]) == string_stress
                push!(stress_vec,parse(Float64,strip(linesplit[3])))
            elseif strip(linesplit[2]) == string_evergreen
                push!(evergreen_vec,parse(Float64,strip(linesplit[3])))
                
            # Next is the leaf on/off status
            elseif strip(linesplit[2]) == string_coh
                push!(leafonoff_vec,parse(Int64,strip(linesplit[3]))) # leaves_off = 1, leaves_on = 2
                
            # Next is the cold and drought statuses
            elseif strip(linesplit[2]) == string_cstatus
                # Cold Leaf Off = 0 or 1, Cold Leaf On = 2
                push!(cstatus_vec,parse(Int64,strip(linesplit[3]))) # phen_cstat_nevercold = 0, phen_cstat_iscold = 1, phen_cstat_notcold = 2
            elseif strip(linesplit[2]) == string_dstatus
                # Drought Leaf On = 2 or 4, Drought Leaf Off = 0 or 1
                push!(dstatus_vec,parse(Int64,strip(linesplit[3]))) # phen_dstat_timeon = 2, phen_dstat_moiston = 3, phen_dstat_moistoff = 0, phen_dstat_timeoff = 1
            end # string match
        end # thread match
    end # eachline
    close(f) # Close the file
end            

Combine the outputs into a dataframe

In [7]:
df = DataFrame(thread=thread_vec,laimemory=laimem_vec,
               leafmass=leafmass_vec,treelai=treelai_vec,
               seasonal_decid=seasonal_vec,stress_decid=stress_vec,evergreen_decid=evergreen_vec,
               leafonoff=leafonoff_vec,cold=cstatus_vec,drought=dstatus_vec);

## Check combinations to determine when `laimemory` is `0`

In [42]:
df_zero = filter(rows -> rows.laimemory .== 0, df);
size(df_zero)

(118001, 10)

In [8]:
df_gbzero = groupby(df,:laimemory);

In [9]:
decid_states = groupby(df,[:stress_decid,:seasonal_decid,:evergreen_decid]);

In [11]:
decid_states[1][1,:]

,thread,laimemory,leafmass,treelai,seasonal_decid,stress_decid,evergreen_decid
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.00467311,0.733959,1.0,0.0,0.0


Deciduous states are mutually exclusive

In [12]:
decid_states[2][1,:]

,thread,laimemory,leafmass,treelai,seasonal_decid,stress_decid,evergreen_decid
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
2,1,0.0,0.0175406,3.2165,0.0,1.0,0.0


In [13]:
decid_states[3][1,:]

,thread,laimemory,leafmass,treelai,seasonal_decid,stress_decid,evergreen_decid
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
4,1,0.0,0.0166008,0.898566,0.0,0.0,1.0


From the above we can see that the deciduous\evergreen states are mutually exclusive

In [20]:
laizero = groupby(df,[:laimemory,:leafonoff])[1];

In [24]:
describe(laizero)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Float64,Real,Float64,Real,Nothing,Nothing
1,thread,35.3023,1,35.0,71,,
2,laimemory,0.0,0.0,0.0,0.0,,
3,leafmass,0.0195217,7.12737e-6,0.0122515,0.244952,,
4,treelai,2.96299,0.0279394,2.64526,15.957,,
5,seasonal_decid,0.164024,0.0,0.0,1.0,,
6,stress_decid,0.499538,0.0,0.0,1.0,,
7,evergreen_decid,0.336438,0.0,0.0,1.0,,
8,leafonoff,2.0,2,2.0,2,,
9,cold,1.16929,0,1.0,2,,
